In [1]:
import numpy as np
import pandas as pd

import requests
import re

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

results = str(requests.get(url).text)

with open('postal_code.txt','w') as output_file:
    output_file.write(results[results.find('<table'):results.find('</table')])

In [58]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
postalcodes = pd.DataFrame(columns=column_names)


    
with open('postal_code.txt','r') as input_file:
    for line in input_file:
        if re.match('<tr>',line) :
            m=re.search('>(.*)<',input_file.readline())
            post_code=m.group(1)
            m=re.search('<td>(<a.*">)?(.*?)(</a>)?</td>',input_file.readline())
            borough=m.group(2)
            m=re.search('<td>(<a.*">)?(.*?)(</a>)?\\n',input_file.readline())
            neighbourhood=m.group(2)
            
            postalcodes=postalcodes.append( {'PostalCode': post_code,
                                             'Borough': borough,
                                             'Neighborhood': neighbourhood}, ignore_index=True)
            
pc_df = postalcodes[postalcodes['Borough'] != 'Not assigned'].reset_index(drop=True)

pc_df.loc[pc_df['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = pc_df['Borough']


pc_fn=pc_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

pc_fn.shape



(103, 3)